# Gemma2-2B Fine-Tuning

## Imports

In [2]:
!pip install transformers peft trl datasets gradio torch
!pip install gemma_pytorch/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 9.6 MB/s eta 0:00:00
ERROR: Invalid requirement: 'gemma_pytorch/': Expected end or semicolon (after name and no valid version specifier)
    gemma_pytorch/
                 ^
Hint: It looks like a path. File 'gemma_pytorch/' does not exist.


In [2]:
!pip install -q kagglehub
!pip install -q ipywidgets

In [3]:
!pip install -q -U immutabledict sentencepiece
!git clone https://github.com/google/gemma_pytorch.git
!mkdir /kaggle/working/gemma/
!mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gemma 0.1 requires sentencepiece==0.2.0, but you have sentencepiece 0.2.1 which is incompatible.
fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.
mkdir: cannot create directory ‘/kaggle/working/gemma/’: No such file or directory
mv: cannot stat '/kaggle/working/gemma_pytorch/gemma/*': No such file or directory


In [4]:
!pip install wandb

In [5]:
import wandb, os

try:
  from kaggle_secrets import UserSecretsClient
  user_secrets = UserSecretsClient()
  os.environ['KAGGLE_KEY'] = user_secrets.get_secret("KAGGLE_KEY")
  os.environ['KAGGLE_USERNAME'] = user_secrets.get_secret("KAGGLE_USERNAME")
  os.environ['WANDB_API_KEY'] = user_secrets.get_secret("WANDB_API_KEY")
except:
  try:
    from google.colab import userdata
    os.environ['KAGGLE_KEY'] = userdata.get("KAGGLE_KEY")
    os.environ['KAGGLE_USERNAME'] = userdata.get("KAGGLE_USERNAME")
    os.environ['WANDB_API_KEY'] = userdata.get("WANDB_API_KEY")
  except:
    print("Could't get secrets")

In [10]:
import huggingface_hub
huggingface_hub.notebook_login()

In [15]:
!pip install --upgrade transformers

In [3]:
import os
import shutil

# Define the path to the Hugging Face cache directory
hf_cache_home = os.path.expanduser("~/.cache/huggingface/hub")

# Check if the cache directory exists and remove it
if os.path.exists(hf_cache_home):
    print(f"Clearing Hugging Face cache at: {hf_cache_home}")
    shutil.rmtree(hf_cache_home)
    print("Hugging Face cache cleared.")
else:
    print("Hugging Face cache directory not found, nothing to clear.")

# Uninstall existing torch, torchvision, and transformers installations
!pip uninstall -y torch torchvision transformers

# Install the latest compatible versions of torch and torchvision
# Explicitly specifying version for torch and torchvision can help prevent conflicts.
# Given the original `torch` version was 2.6.0, I will try to reinstall with a compatible `torchvision`.
!pip install torch torchvision --upgrade

# Reinstall transformers to ensure compatibility with the new torch/torchvision
!pip install transformers --upgrade

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import huggingface_hub

# Log in to Hugging Face (if not already logged in, this will prompt)
huggingface_hub.notebook_login()

model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    attn_implementation="eager" # Explicitly use eager attention to avoid XLA issues
)

print("Model and tokenizer loaded successfully.")

Hugging Face cache directory not found, nothing to clear.
Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.8 MB/s

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Model and tokenizer loaded successfully.


## Baseline sample tests

First, making sure it generates something

In [13]:
prompt = "If you have 5 apples but someone takes 2, how many do you have left? Show your work, then write your final answer on the last line after ####"
inputs = tokenizer(prompt, return_tensors="pt").to("cpu")
outputs = model.generate(**inputs, max_new_tokens=100, do_sample=False)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

NameError: name 'model' is not defined

In [12]:
import json

# Define the path to the test.json file
json_file_path = '/test.json'

# Initialize an empty list for prompts
prompts = []

try:
    # Open the test.json file in read mode
    with open(json_file_path, 'r') as f:
        # Read the entire content of the file
        file_content = f.read()
        # Parse the content as a JSON object
        test_data = json.loads(file_content)

    # Extract the list of prompts
    # This assumes test_data is a list of dictionaries, each with a 'prompt' key
    if isinstance(test_data, list):
        prompts = [item['question'] for item in test_data if 'question' in item]
    else:
        print("The JSON structure is not a list; could not extract prompts.")

    print(f"Successfully loaded {len(prompts)} prompts from {json_file_path}")
    # Optionally, print the first few prompts to verify
    if prompts:
        print("First 3 prompts:")
        for i, p in enumerate(prompts[:3]):
            print(f"  {i+1}. {p}")

except FileNotFoundError:
    print(f"Error: The file '{json_file_path}' was not found. Please ensure it exists in the current directory.")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from '{json_file_path}'. Please check the file's format.")
except KeyError:
    print(f"Error: 'prompt' key not found in some items within '{json_file_path}'. Please check the JSON structure.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Successfully loaded 10 prompts from /test.json
First 3 prompts:
  1. A plant is 10 cm tall. It grows 15% in one week. How tall is it after the growth?
  2. Calculate the product: 234 × 6.
  3. There are 500 seats in a theater. 378 tickets were sold. How many seats remain unsold?


In [13]:
model_outputs = []
device = "cpu" # Set device to CPU to avoid CUDA issues

for i, prompt_text in enumerate(prompts):
    print(f"\n--- Processing Prompt {i+1} ---")
    print(f"Original Prompt: {prompt_text}")

    # Tokenize the prompt and move to CPU
    input_ids = tokenizer.encode(prompt_text, return_tensors="pt").to(device)

    # Generate a response from the model
    outputs = model.generate(input_ids, max_new_tokens=100, do_sample=False)

    # Decode the generated output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Generated Output: {decoded_output}")

    model_outputs.append({"prompt": prompt_text, "generated_response": decoded_output})

print("\nAll prompts processed.")


--- Processing Prompt 1 ---
Original Prompt: A plant is 10 cm tall. It grows 15% in one week. How tall is it after the growth?
Generated Output: A plant is 10 cm tall. It grows 15% in one week. How tall is it after the growth?

The plant will be 10 cm * (115/100) = 11.5 cm tall after the growth.

--- Processing Prompt 2 ---
Original Prompt: Calculate the product: 234 × 6.
Generated Output: Calculate the product: 234 × 6.

Sure, here's the calculation:

234 × 6 = 1304

--- Processing Prompt 3 ---
Original Prompt: There are 500 seats in a theater. 378 tickets were sold. How many seats remain unsold?
Generated Output: There are 500 seats in a theater. 378 tickets were sold. How many seats remain unsold?

**Solution:**

To find the number of seats remaining unsold, we can subtract the number of tickets sold from the total number of seats.

500 seats - 378 tickets = 122 seats

Therefore, there are 122 seats remaining unsold in the theater.

--- Processing Prompt 4 ---
Original Prompt: A cy

Quantatively, we can see that answer accuracy is:

## MATH dataset

In [4]:
from datasets import load_dataset

ds = load_dataset("qwedsacf/competition_math")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-7320a6f3aba8eb(…):   0%|          | 0.00/4.85M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [5]:
ds_split = ds['train'].train_test_split(test_size=0.10, seed=2025)

train_val_split = ds_split['train'].train_test_split(test_size=0.1111, seed=2025)

math_train = train_val_split['train']
math_val   = train_val_split['test']
math_test  = ds_split['test']

print("Train size:", len(math_train))
print("Val size:",   len(math_val))
print("Test size:",  len(math_test))

Train size: 10000
Val size: 1250
Test size: 1250


In [4]:
print(math_train[0])

{'problem': 'Let $a,$ $b,$ and $c$ be distinct real numbers such that\n\\[\\frac{a^3 + 6}{a} = \\frac{b^3 + 6}{b} = \\frac{c^3 + 6}{c}.\\]Find $a^3 + b^3 + c^3.$', 'level': 'Level 4', 'type': 'Intermediate Algebra', 'solution': "Let\n\\[k = \\frac{a^3 + 6}{a} = \\frac{b^3 + 6}{b} = \\frac{c^3 + 6}{c}.\\]Then $a,$ $b,$ and $c$ are all roots of\n\\[k = \\frac{x^3 + 6}{x},\\]or $x^3 - kx + 6 = 0.$  By Vieta's formulas, $a + b + c = 0.$\n\nAlso,\n\\begin{align*}\na^3 - ka + 6 &= 0, \\\\\nb^3 - kb + 6 &= 0, \\\\\nc^3 - kc + 6 &= 0.\n\\end{align*}Adding these, we get $a^3 + b^3 + c^3 - k(a + b + c) + 18 = 0,$ so $a^3 + b^3 + c^3 = k(a + b + c) - 18 = \\boxed{-18}.$"}


In [5]:
print(math_train[1])
print(math_train[2])
print(math_train[3])

{'problem': 'A ball travels on a parabolic path in which the height (in feet) is given by the expression $-25t^2+75t+24$, where $t$ is the time after launch. At what time is the height of the ball at its maximum?', 'level': 'Level 5', 'type': 'Algebra', 'solution': 'First, we find the maximum height of the ball by maximizing the expression $-25t^2+75t+24$. We will do this by completing the square. Factoring a $-25$ from the first two terms, we have  \\[-25t^2+75t+24=-25(t^2-3t)+24\\]To complete the square, we add and subtract $\\left( -\\frac{3}{2}\\right)^2=\\frac{9}{4}$ inside the parentheses to get \\begin{align*}\n-25(t^2-3t)+24&=-25\\left(t^2-3t+\\frac{9}{4}-\\frac{9}{4}\\right)+24\\\\\n&=-25\\left(\\left(t-\\frac{3}{2}\\right)^2-\\frac{9}{4}\\right)+24\\\\\n&=-25\\left(t-\\frac{3}{2}\\right)^2+\\frac{225}{4}+\\frac{96}{4}\\\\\n&=-25\\left(t-\\frac{3}{2}\\right)^2+\\frac{321}{4}\n\\end{align*}Since $-25\\left(t-\\frac{3}{2}\\right)^2$ is always non-positive, the maximum value of t

The inspection of `math_train` entries confirmed the presence of numerous LaTeX commands and special characters such as `$`, `\`, `\begin{align*}` (and `\end{align*}`), `\\`, `\cdot`, `\ldots`, `\frac`, `\ge`, `\ge`, `\boxed`, `\binom`, `\times`, `\%`, and others. These need to be removed or replaced to make the text more readable and suitable for natural language processing tasks.

Now, let's create a cleaning function to handle these patterns. We'll start with a basic function and refine it as needed.

**Reasoning**:
Based on the observed patterns of LaTeX commands and special characters, I will now create a Python function to clean the text data. This function will use regular expressions to remove or replace these unwanted elements, making the text suitable for further processing.



## Cleaning MATH dataset

In [6]:
import re

# --- Utility: remove surrounding spaces ---
def _strip(s):
    return s.strip()

# --- Step 1: Convert LaTeX math syntax to plain math BEFORE tagging ---
def _normalize_latex(expr):

    # Fractions
    expr = re.sub(r'\\frac\s*\{([^}]+)\}\s*\{([^}]+)\}', r'(\1)/(\2)', expr)

    # Binomials
    expr = re.sub(r'\\binom\s*\{([^}]+)\}\s*\{([^}]+)\}', r'C(\1, \2)', expr)

    # Square roots
    expr = re.sub(r'\\sqrt\s*\{([^}]+)\}', r'sqrt(\1)', expr)

    # Operators
    replacements = {
        r'\\cdot': '*',
        r'\\times': '*',
        r'\\ge': '>=',
        r'\\le': '<=',
        r'\\neq': '!=',
        r'\\approx': '~',
        r'\\equiv': '==',
        r'\\infty': 'infinity',
        r'\\ldots': '...',
        r'\\sum': 'sum',
        r'\\prod': 'product',
    }
    for key, val in replacements.items():
        expr = re.sub(key, val, expr)

    # Greek letters
    greek = [
        'alpha', 'beta', 'gamma', 'delta', 'epsilon', 'theta', 'lambda',
        'mu', 'nu', 'pi', 'rho', 'sigma', 'tau', 'phi', 'chi', 'psi', 'omega'
    ]
    for g in greek:
        expr = re.sub(rf'\\{g}', g, expr)

    # Formatting removal
    expr = re.sub(r'\\text\{([^}]+)\}', r'\1', expr)
    expr = re.sub(r'\\boxed\{([^}]+)\}', r'\1', expr)

    # Remove \left, \right
    expr = re.sub(r'\\left', '', expr)
    expr = re.sub(r'\\right', '', expr)

    # Remove stray command backslashes
    expr = re.sub(r'\\([a-zA-Z]+)', '', expr)
    expr = expr.replace('\\', '')

    return expr.strip()


# --- Step 2: Wrap math expressions in [MATH ...] spans ---
def _wrap_math(text):

    # Inline $...$
    def inline_repl(m):
        expr = _normalize_latex(m.group(1))
        return f"[MATH {expr}]"

    text = re.sub(r'\$(.+?)\$', inline_repl, text)

    # Display \[...\]
    def display_repl(m):
        expr = _normalize_latex(m.group(1))
        return f"[MATH {expr}]"

    text = re.sub(r'\\\[(.+?)\\\]', display_repl, text, flags=re.DOTALL)

    # Parentheses math \(...\)
    text = re.sub(r'\\\((.+?)\\\)', inline_repl, text)

    return text


# --- Step 3: Handle align, gather, eqnarray blocks ---
def _replace_align_envs(text):

    pattern = r'\\begin\{(align\*?|gather\*?|eqnarray\*?)\}(.+?)\\end\{\1\}'

    def repl(m):
        block = m.group(2)

        # Split on line breaks or & alignment markers
        lines = re.split(r'\\\\', block)
        cleaned = []
        for line in lines:
            line = line.replace('&', '')
            line = _normalize_latex(line)
            if line.strip():
                cleaned.append(f"[MATH {line.strip()}]")
        return " ".join(cleaned)

    return re.sub(pattern, repl, text, flags=re.DOTALL)


# --- FINAL CLEAN FUNCTION ---
def clean_text(text):

    text = _replace_align_envs(text)
    text = _wrap_math(text)

    # Remove duplicate spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Fix malformed MATH blocks like "[MATH x]" or "[MATHx]" -> "[MATH x ]"
    text = re.sub(r'\[MATH([^A-Za-z0-9])', r'[MATH \1', text)

    # Ensure closing brackets always have at least 1 space before ]
    text = re.sub(r'\[MATH\s+([^\]]*?)\]', lambda m: f"[MATH {m.group(1).strip()} ]", text)

    return text.strip()

In [31]:
clean_text("Let\n\\[k = \\frac{a^3 + 6}{a} = \\frac{b^3 + 6}{b} = \\frac{c^3 + 6}{c}.\\]Then $a,$ $b,$ and $c$ are all roots of\n\\[k = \\frac{x^3 + 6}{x},\\]or $x^3 - kx + 6 = 0.$  By Vieta's formulas, $a + b + c = 0.$\n\nAlso,\n\\begin{align*}\na^3 - ka + 6 &= 0, \\\\\nb^3 - kb + 6 &= 0, \\\\\nc^3 - kc + 6 &= 0.\n\\end{align*}Adding these, we get $a^3 + b^3 + c^3 - k(a + b + c) + 18 = 0,$ so $a^3 + b^3 + c^3 = k(a + b + c) - 18 = \\boxed{-18}.$")

"Let [MATH k = (a^3 + 6)/(a) = (b^3 + 6)/(b) = (c^3 + 6)/(c). ]Then [MATH a, ] [MATH b, ] and [MATH c ] are all roots of [MATH k = (x^3 + 6)/(x), ]or [MATH x^3 - kx + 6 = 0. ] By Vieta's formulas, [MATH a + b + c = 0. ] Also, [MATH a^3 - ka + 6 = 0, ] [MATH b^3 - kb + 6 = 0, ] [MATH c^3 - kc + 6 = 0. ]Adding these, we get [MATH a^3 + b^3 + c^3 - k(a + b + c) + 18 = 0, ] so [MATH a^3 + b^3 + c^3 = k(a + b + c) - 18 = -18. ]"

In [7]:
#applying to training/testing splits
math_train = math_train.map(lambda example: {'cleaned_problem': clean_text(example['problem'])})
math_train = math_train.map(lambda example: {'cleaned_solution': clean_text(example['solution'])})

math_test = math_test.map(lambda example: {'cleaned_problem': clean_text(example['problem'])})
math_test = math_test.map(lambda example: {'cleaned_solution': clean_text(example['solution'])})

math_val = math_val.map(lambda example: {'cleaned_problem': clean_text(example['problem'])})
math_val = math_val.map(lambda example: {'cleaned_solution': clean_text(example['solution'])})


print("Cleaning applied to math_train, math_test and math_val datasets.")
print("Sample of math_train after cleaning:")
print(math_train[0]['problem'])
print(math_train[0]['cleaned_problem'])
print(math_train[0]['solution'])
print(math_train[0]['cleaned_solution'])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Cleaning applied to math_train, math_test and math_val datasets.
Sample of math_train after cleaning:
What is the distance between the two intersections of $y=x^2$ and $x+y=1$?
What is the distance between the two intersections of [MATH y=x^2 ] and [MATH x+y=1 ]?
To find the $x$-coordinates of the intersections, substitute $x^2$ for $y$ in $x+y=1$ and solve for $x$, resulting in  \begin{align*}
x+x^2&=1 \\
\Rightarrow \qquad x^2+x-1&=0 \\
\Rightarrow \qquad x&=\frac{-1\pm\sqrt{1+4}}2=\frac{-1\pm\sqrt5}2\\
\end{align*}Using each of these coordinates to solve for $y$ gives us the intersections at $\left(\frac{-1+\sqrt5}2,\frac{3-\sqrt5}2\right)$ and $\left(\frac{-1-\sqrt5}2,\frac{3+\sqrt5}2\right)$.  Using the distance formula, we have \begin{align*}
&\sqrt{ \left(\frac{-1+\sqrt5}{2}-\frac{-1-\sqrt5}{2}\right)^2 + \left(\frac{3-\sqrt5}2-\frac{3+\sqrt5}2\right)^2 }\\
&\qquad=\sqrt{\left(\frac{2\sqrt5}2\right)^2 + \left(-\frac{2\sqrt5}2\right)^2}\\
&\qquad=\sqrt{ 2\sqrt5^2 }\\
&\qquad=\box

In [13]:
math_train[0]

{'problem': 'Let $a,$ $b,$ and $c$ be distinct real numbers such that\n\\[\\frac{a^3 + 6}{a} = \\frac{b^3 + 6}{b} = \\frac{c^3 + 6}{c}.\\]Find $a^3 + b^3 + c^3.$',
 'level': 'Level 4',
 'type': 'Intermediate Algebra',
 'solution': "Let\n\\[k = \\frac{a^3 + 6}{a} = \\frac{b^3 + 6}{b} = \\frac{c^3 + 6}{c}.\\]Then $a,$ $b,$ and $c$ are all roots of\n\\[k = \\frac{x^3 + 6}{x},\\]or $x^3 - kx + 6 = 0.$  By Vieta's formulas, $a + b + c = 0.$\n\nAlso,\n\\begin{align*}\na^3 - ka + 6 &= 0, \\\\\nb^3 - kb + 6 &= 0, \\\\\nc^3 - kc + 6 &= 0.\n\\end{align*}Adding these, we get $a^3 + b^3 + c^3 - k(a + b + c) + 18 = 0,$ so $a^3 + b^3 + c^3 = k(a + b + c) - 18 = \\boxed{-18}.$",
 'cleaned_problem': 'Let [MATH a,] [MATH b,] and [MATH c] be distinct real numbers such that [MATH (a^3 + 6)/(a) = (b^3 + 6)/(b) = (c^3 + 6)/(c).]Find [MATH a^3 + b^3 + c^3.]',
 'cleaned_solution': "Let [MATH k = (a^3 + 6)/(a) = (b^3 + 6)/(b) = (c^3 + 6)/(c).]Then [MATH a,] [MATH b,] and [MATH c] are all roots of [MATH k = (x

In [8]:
import re
import torch
from tqdm import tqdm

def extract_answer(text):
    """
    Extract the final numeric answer from model output.
    Priority:
        1. Last non-empty [MATH ...] block
        2. Last \boxed{...}
        3. Last number ONLY IF NO math/boxed blocks found
    """

    # 1. Extract full MATH blocks: [MATH 42], [MATH -7], etc.
    blocks = re.findall(r'\[MATH\s+([^\]]+)\]', text)
    if blocks:  # only non-empty matches
        last = blocks[-1].strip()
        candidates = re.findall(r'-?\d+(?:_\d+)?(?:\.\d+)?', last)
        if candidates:
            return candidates[-1]

    # 2. Boxed answers
    boxed = re.findall(r'\\boxed\{([^}]*)\}', text)
    if boxed:
        candidates = re.findall(r'-?\d+(?:_\d+)?(?:\.\d+)?', boxed[-1])
        if candidates:
            return candidates[-1]

    # 3. LAST RESORT — only if no MATH or boxed answer found
    #    This prevents greedy extraction of numbers from the explanation
    return None



def eval_math(model, tokenizer, data, max_new_tokens=128, verbose=False, num_eval=None):
    """Evaluate model accuracy on your cleaned math dataset."""
    model.eval()

    prompt_template = """Solve this math problem. Show your work and put your final answer inside a [MATH ...] block.

Question: {question}
Answer:"""

    correct = 0
    formatted = 0
    if num_eval is None:
        num_eval = len(data)

    for i in tqdm(range(num_eval), disable=verbose):
        item = data[i]

        # Ground truth answer
        gt_answer = extract_answer(item["cleaned_solution"])

        # Prompt
        prompt = prompt_template.format(question=item["cleaned_problem"])
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
            )

        # Extract generated text
        generated_ids = outputs[0][inputs["input_ids"].shape[1]:]
        generated = tokenizer.decode(generated_ids, skip_special_tokens=True)

        pred_answer = extract_answer(generated)

        # Verbose output
        if verbose:
            print(f"Problem {i}\n{item['cleaned_problem']}\n")
            print("Real solution:", item["cleaned_solution"])
            print("Ground Truth:", gt_answer)
            print("Generated output:", generated)
            print("Pred Answer:", pred_answer)
            print("\n" + "-"*50 + "\n")

        # Format and accuracy checks
        if pred_answer is not None:
            formatted += 1
            if pred_answer == gt_answer:
                correct += 1

    total = num_eval
    format_rate = formatted / total
    accuracy = correct / formatted if formatted else 0.0

    print(f"Format rate: {format_rate:.2%} ({formatted}/{total})")
    print(f"Accuracy:    {accuracy:.2%} ({correct}/{formatted})")

    return format_rate, accuracy


In [38]:
#testing eval function
eval_math(model, tokenizer, math_train, verbose=True, num_eval=3)

Problem 0
Let [MATH a, ] [MATH b, ] and [MATH c ] be distinct real numbers such that [MATH (a^3 + 6)/(a) = (b^3 + 6)/(b) = (c^3 + 6)/(c). ]Find [MATH a^3 + b^3 + c^3. ]

Real solution: Let [MATH k = (a^3 + 6)/(a) = (b^3 + 6)/(b) = (c^3 + 6)/(c). ]Then [MATH a, ] [MATH b, ] and [MATH c ] are all roots of [MATH k = (x^3 + 6)/(x), ]or [MATH x^3 - kx + 6 = 0. ] By Vieta's formulas, [MATH a + b + c = 0. ] Also, [MATH a^3 - ka + 6 = 0, ] [MATH b^3 - kb + 6 = 0, ] [MATH c^3 - kc + 6 = 0. ]Adding these, we get [MATH a^3 + b^3 + c^3 - k(a + b + c) + 18 = 0, ] so [MATH a^3 + b^3 + c^3 = k(a + b + c) - 18 = -18. ]
Ground Truth: -18
Generated output:  [MATH 9a^3 + 9b^3 + 9c^3. ]

**Work:**

Let [MATH a, b, c ] be distinct real numbers such that [MATH (a^3 + 6)/(a) = (b^3 + 6)/(b) = (c^3 + 6)/(c). ]

Then,

[MATH a^3 + 6 = (a^3 + 6)/(a) ]

[MATH b^3 + 6 = (b^3 + 6)/(b) ]

[MATH c^3 +
Pred Answer: 6

--------------------------------------------------

Problem 1
A ball travels on a parabolic path in 

(0.6666666666666666, 0.0)

# SFT training

## Format dataset questions


In [9]:
def format_math(example, tokenizer):
    """Format example for training with proper label masking.

    Args:
        example: Dictionary with 'cleaned_problem' and 'cleaned_solution' keys
        tokenizer: HuggingFace tokenizer

    Returns:
        Dictionary with 'input_ids', 'attention_mask', and 'labels'
        Labels are -100 for question tokens (ignored in loss) and
        actual token IDs for answer tokens.
    """
    prompt_template = """Solve the math problem; show your work in a "Reasoning:" section.
You MUST finish with a single line that starts exactly with:

Final Answer:
<single token with final answer, e.g. -18, 42_7, 3/2, 1.25>

Rules:
- The Final Answer line must contain *only* the final answer (no text, no units, no brackets, no LaTeX).
- Do NOT include any additional text after the Final Answer line.
- Reasoning may include math or text, but the final answer must be plain.

Question:
{question}

Reasoning:
"""
    question_text = prompt_template.format(question=example['cleaned_problem'])
    answer_text = f"{example['cleaned_solution']}"

    # Tokenize question and answer separately
    question_tokens = tokenizer(question_text, add_special_tokens=True)
    answer_tokens = tokenizer(answer_text, add_special_tokens=False)

    # Combine
    input_ids = question_tokens['input_ids'] + answer_tokens['input_ids']
    attention_mask = question_tokens['attention_mask'] + answer_tokens['attention_mask']

    # Create labels: -100 for question (ignored), actual tokens for answer
    labels = [-100] * len(question_tokens['input_ids']) + answer_tokens['input_ids']

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

# Example:
print(f"First Question Tokenized: {tokenizer(math_train[0]['cleaned_problem'], add_special_tokens=True)}")
print()
print(f"First Answer Tokenized: {tokenizer(math_train[0]['cleaned_solution'], add_special_tokens=True)}")
print()
example_formatted = format_math(math_train[0], tokenizer)
print(f"Combined tokenized input: {example_formatted['input_ids']}")
print()
print(f"Prediction targets/labels: {example_formatted['labels']}")

First Question Tokenized: {'input_ids': [2, 1841, 603, 573, 6075, 1865, 573, 1378, 94687, 576, 892, 70025, 597, 235293, 235297, 235393, 235284, 5896, 578, 892, 70025, 1141, 235340, 235267, 235293, 235274, 5896, 235336], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

First Answer Tokenized: {'input_ids': [2, 1469, 1717, 573, 892, 70025, 1141, 5896, 235290, 44394, 576, 573, 94687, 235269, 28812, 892, 70025, 1141, 235393, 235284, 5896, 604, 892, 70025, 597, 5896, 575, 892, 70025, 1141, 235340, 235267, 235293, 235274, 5896, 578, 11560, 604, 892, 70025, 1141, 11994, 13257, 575, 892, 70025, 1141, 235340, 235297, 235393, 235284, 235293, 235274, 5896, 892, 70025, 1141, 235393, 235284, 235340, 235297, 235290, 235274, 235293, 235276, 5896, 892, 70025, 1141, 2549, 235290, 235274, 235278, 235274, 235340, 235310, 4596, 235284, 2549, 235290, 235274, 235308, 235270, 235284, 5896, 15325, 1853, 576, 1450, 23706, 577, 11560, 604, 892, 70025, 597,

## Train model

In [10]:
# You do not need to modify this code
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, TrainerCallback, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, TaskType
import torch

# Define a simple callback to print the train loss periodically
class PrintLossCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and 'loss' in logs:
            print(f"Step {state.global_step}: Train loss={logs['loss']:.4f}")

def train_math(model, tokenizer, train_data, val_data, test_data,
                batch_size, gradient_accumulation_steps,
                learning_rate, max_epochs, patience,
                lora_r, lora_drop):
    """Fine-tune model with LoRA on MATH."""

    # Set padding token
    tokenizer.pad_token = tokenizer.eos_token

    # Apply LoRA
    lora_config = LoraConfig(r=lora_r, lora_alpha=lora_r * 2,
                             target_modules=["q_proj", "k_proj", "v_proj"],
                             lora_dropout=lora_drop, bias="none",
                             task_type=TaskType.CAUSAL_LM)
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()

    # Format and tokenize with proper masking
    train_tokenized = train_data.map(
        lambda x: format_math(x, tokenizer),
        remove_columns=train_data.column_names
    )
    val_tokenized = val_data.map(
        lambda x: format_math(x, tokenizer),
        remove_columns=val_data.column_names
    )

    # Data collator for padding with label padding
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, label_pad_token_id=-100, padding=True)

    # Training arguments
    args = TrainingArguments(
        output_dir="./math_checkpoints",
        num_train_epochs=max_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        logging_steps=20,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        fp16=True,
        bf16=False,  #edit: true for TPU usage
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_tokenized,
        eval_dataset=val_tokenized,
        data_collator=data_collator,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=patience), PrintLossCallback()],
    )

    # Train
    trainer.train()

    # Final evaluation
    print("\nFinal test results:")
    eval_math(model, tokenizer, math_test)

    return model

## Setting hyperparameters

In [45]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

In [17]:
# TODO: Set your hyperparameters here

batch_size = 2  # Try 2, 4, or 8
gradient_accumulation_steps = 4  # Try 4 or 8
learning_rate = 5e-5 # Try 1e-4, 5e-5, or 1e-5
max_epochs = 5  # Try 5-10
patience = 2  # Try 2 or 3
lora_r = 32  # Try 8, 16, or 32
lora_drop =0.05  # Try 0.05 or 0.1

# Free GPU memory and reload a fresh model for fine-tuning
# This is to ensure that if you run this cell multiple times
# with different hyperparameters, you always start from the
# clean original pretrained model (not a partially fine-tuned one)
import gc
del model
gc.collect()
torch.cuda.empty_cache()

#tpu_device = xm.xla_device()
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    torch_dtype=torch.float16,
    device_map="cuda"
)
print("Fresh model loaded and ready for fine-tuning")


# Run training
model = train_math(
    model=model,
    tokenizer=tokenizer,
    train_data=math_train,
    val_data=math_val,
    test_data=math_test,
    batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    max_epochs=max_epochs,
    patience=patience,
    lora_r=lora_r,
    lora_drop=lora_drop
)

NameError: name 'model' is not defined